# A.3 Pair programming
https://codebook.xanadu.ai/A.3

In [1]:
import numpy as np
import pennylane as qml

## Codercise A.3.1

In [2]:
def oracle_matrix(combo):
    """
    returns: 
        array[float]: oracle matrix for a secret combination
    ****************
    combo (list[int]): the secret combination, given as list of bits
    """
    n = len(combo)
    index = np.ravel_multi_index(combo, [2]*n) # index of solution
    my_array = np.identity(2**n) # create identity matrix
    my_array[index, index] = -1
    return my_array

In [3]:
n_bits = 4
dev = qml.device("default.qubit", wires=n_bits)

@qml.qnode(dev)
def pair_circuit(x_tilde, combo):
    """
    returns: 
        array[float]: probabilities of measuring 0 and 1 on the last qubit
    ****************
    x_tilde (list [int]): (n-1)-bit string that labels the pairs of the n-bit strings
    combo (list[int]): the secret combination, given as list of bits
    """
    for i in range(n_bits - 1):
        if x_tilde[i] == 1:
            qml.PauliX(wires = i)
    qml.Hadamard(wires = n_bits - 1)
    qml.QubitUnitary(oracle_matrix(combo), wires = range(n_bits))
    qml.Hadamard(wires = n_bits - 1)
    return qml.probs(wires = n_bits - 1)
    

## Codercise A.3.2

In [4]:
def secret_combo(n_bits):
    """
    returns:
        array[int]: n_bit-dimensional array of randomly generated 0's and 1's
    ****************
    n_bits (int): number of bits in the secret combination
    """
    combo_list = np.empty(n_bits, dtype=int)
    for i in range(n_bits):
        combo_list[i] = np.random.randint(0, high=2, dtype=int)
    return(combo_list)

In [5]:
def pair_lock_picker(trials):
    """
    returns:
        int: the average number of pair tests needed for a given number of trials
    ****************
    trials (int): number of times to test
    """
    x_tilde_strs = [np.binary_repr(n, n_bits-1) for n in range(2**(n_bits-1))]
    x_tildes = [[int(s) for s in x_tilde_str] for x_tilde_str in x_tilde_strs] 

    test_numbers = []

    for trial in range(trials):
        combo = secret_combo(n_bits)
        counter = 0
        for x_tilde in x_tildes:
            counter += 1
            if np.isclose(pair_circuit(x_tilde, combo)[1], 1):
                # if the probability of measuring 1 on the last qubit is close to 1, exit loop
                break
        test_numbers.append(counter)
    return sum(test_numbers)/trials

trials = 500
output = pair_lock_picker(trials)

print(f"For {n_bits} bits, it takes", output, "pair tests on average.")    

For 4 bits, it takes 4.718 pair tests on average.
